In [1]:
from count_stops import *
import os

In [2]:
FILENAME = os.environ["FILENAME"] #"Sample_Ldn_100121_rep1" - comment in/out this line if not using this script on
# the command line
ERROR_CUTOFF = 0.02

# read in the vcf file
df = pd.read_csv(
    f'../data/ww_dump/plate_validation/{FILENAME}/'+
    f'{FILENAME}.vcf', sep="\t", skiprows=22)

# extract the relevant columns, skipping indels, only interested in substitutions compared to the wuhan ref
df['DEPTH'] = df['INFO'].apply(
    lambda x: (0 if x.split(";")[0].split("=")[-1] == "INDEL" else int(x.split(";")[0].split("=")[-1])))
df['COUNTS'] = df['INFO'].apply(lambda x: [int(y) for y in x.split(";")[1].split("=")[-1].split(",")])
df['ALTS'] = df['ALT'].apply(lambda x: x.split(','))
df['ALT_COUNTS'] = df['COUNTS'].apply(lambda x: sum(x[1:]))
df['ERROR_PC'] = df['ALT_COUNTS']/df['DEPTH']
df['ALLELES'] = df.apply(lambda x: (x.REF + "," + x.ALT).split(","), axis=1)
df['ALLELES_DICT'] = df.apply(lambda x: dict(zip(x.ALLELES, x.COUNTS)), axis=1)
df['MUT_TO_A'] = df.apply(lambda x: 0 if x.REF == 'A' or not 'A' in x.ALLELES_DICT else x.ALLELES_DICT['A'], axis=1)
df['MUT_TO_C'] = df.apply(lambda x: 0 if x.REF == 'C' or not 'C' in x.ALLELES_DICT else x.ALLELES_DICT['C'], axis=1)
df['MUT_TO_G'] = df.apply(lambda x: 0 if x.REF == 'G' or not 'G' in x.ALLELES_DICT else x.ALLELES_DICT['G'], axis=1)
df['MUT_TO_T'] = df.apply(lambda x: 0 if x.REF == 'T' or not 'T' in x.ALLELES_DICT else x.ALLELES_DICT['T'], axis=1)
df = pd.DataFrame(
    df[['#CHROM','POS', 'REF', 'DEPTH', 'MUT_TO_A', 'MUT_TO_C', 'MUT_TO_G', 'MUT_TO_T', 'ERROR_PC', 'ALT_COUNTS']])

df["A_STOP"] = df.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "A", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
df["C_STOP"] = df.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "C", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
df["G_STOP"] = df.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "G", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
df["T_STOP"] = df.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "T", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
df["ANY_STOP"] = df.apply(lambda x: int(x.A_STOP + x.C_STOP + x.G_STOP + x.T_STOP), axis=1)

# We end up with a dataframe with the following columns:
# CHROM (name of the reference genome, MN908947.3)
# POS (genome position, 1 indexed)
# REF (reference nucleotide)
# DEPTH (sequencing depth of coverage)
# MUT_TO_A/C/G/T (a count of how many reads had a mutation to this nucleotide, it's a 0 if it's the same 
# nucleotide as the reference by default)
# ERROR_PC (% error, but actually only between 0 and 1)
# ALT_COUNTS (total count of non-reference nucleotides at that position)
# A/C/G/T_STOP (a flag telling you whether a mutation at that position would cause a stop codon)
# ANY_STOP (a flag - how many possible mutations at that position could cause a stop codon)
df.head()

,#CHROM,POS,REF,DEPTH,MUT_TO_A,MUT_TO_C,MUT_TO_G,MUT_TO_T,ERROR_PC,ALT_COUNTS,A_STOP,C_STOP,G_STOP,T_STOP,ANY_STOP
0,MN908947.3,31,A,1,0,0,0,0,0.0,0,0,0,0,0,0
1,MN908947.3,32,C,1,0,0,0,0,0.0,0,0,0,0,0,0
2,MN908947.3,33,C,1,0,0,0,0,0.0,0,0,0,0,0,0
3,MN908947.3,34,A,1,0,0,0,0,0.0,0,0,0,0,0,0
4,MN908947.3,35,A,1,0,0,0,0,0.0,0,0,0,0,0,0


In [3]:
# take a subset of positions where:
# - depth 10 or higher
df = df.where(df["DEPTH"] >= 10).dropna()

In [4]:
# to compute PCR errors we will only look at site rate mutation to stop codons - and only at
# certain points in the genome, specifically:
# - within ORF1a or ORF1b or the S gene
# - not within 100bps of the end of the gene

stop_codon_subset = df.where(
    ((df["POS"] > 300) & (df["POS"] < 12000)) | 
    ((df["POS"] > 13500) & (df["POS"] < 20000)) | 
    ((df["POS"] > 21600) & (df["POS"] < 25000))
).dropna()


# out of this subset of data we are interested in how many sites have a high/low rate of errors (sequencing vs pcr)
# and how many could mutate to a stop codon
stop_codon_nuc_counts = stop_codon_subset["REF"].value_counts()
stop_codon_stop_counts = stop_codon_subset[["REF", "A_STOP", "C_STOP", "G_STOP", "T_STOP", "ANY_STOP"]].groupby("REF").sum()

In [5]:
# count how many sites have a PCR error - to do this we need to look at 'high frequency errors' but 
# specifically count mutations to stop codons, as these are for sure not real SNPs. 
x1 = stop_codon_subset.where(stop_codon_subset["ERROR_PC"] >= ERROR_CUTOFF).dropna()

x1["TO_A"] = x1.apply(lambda y: 1 if y.MUT_TO_A/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_A > 5 else 0 ,axis=1)
x1["TO_C"] = x1.apply(lambda y: 1 if y.MUT_TO_C/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_C > 5 else 0 ,axis=1)
x1["TO_G"] = x1.apply(lambda y: 1 if y.MUT_TO_G/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_G > 5 else 0 ,axis=1)
x1["TO_T"] = x1.apply(lambda y: 1 if y.MUT_TO_T/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_T > 5 else 0 ,axis=1)
x1["COUNT"] = 1
x1["A_STOP"] = x1.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "A", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
x1["C_STOP"] = x1.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "C", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
x1["G_STOP"] = x1.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "G", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
x1["T_STOP"] = x1.apply(lambda x: (1 if is_stop_codon(int(x['POS']), "T", reference, orf_starts, orf_ends, orf_frames) else 0), axis=1)
x1["ANY_STOP"] = x1.apply(lambda x: int(x.A_STOP + x.C_STOP + x.G_STOP + x.T_STOP), axis=1)

x2 = x1[["REF", "TO_A", "TO_C", "TO_G", "TO_T"]]
x2["TO_A_STOP"] = x1.apply(lambda x: int(x.A_STOP * x.TO_A), axis=1)
x2["TO_C_STOP"] = x1.apply(lambda x: int(x.C_STOP * x.TO_C), axis=1)
x2["TO_G_STOP"] = x1.apply(lambda x: int(x.G_STOP * x.TO_G), axis=1)
x2["TO_T_STOP"] = x1.apply(lambda x: int(x.T_STOP * x.TO_T), axis=1)
x2 = x2.groupby("REF").sum()
pcr_errors = x2

pcr_errors = pd.merge(pcr_errors, stop_codon_nuc_counts, left_index=True, right_index=True)
pcr_errors = pd.merge(pcr_errors, stop_codon_stop_counts, left_index=True, right_index=True)
pcr_errors = pcr_errors.rename(columns={"REF": "NUC_COUNT"})
pcr_errors.head()

<ipython-input-1-55ec3ba9c051>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2["TO_A_STOP"] = x1.apply(lambda x: int(x.A_STOP * x.TO_A), axis=1)
<ipython-input-1-55ec3ba9c051>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2["TO_C_STOP"] = x1.apply(lambda x: int(x.C_STOP * x.TO_C), axis=1)
<ipython-input-1-55ec3ba9c051>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,TO_A,TO_C,TO_G,TO_T,TO_A_STOP,TO_C_STOP,TO_G_STOP,TO_T_STOP,NUC_COUNT,A_STOP,C_STOP,G_STOP,T_STOP,ANY_STOP
C,1,0,0,3,0,0,0,0,95,5.0,0.0,4.0,13.0,22.0
G,0,0,0,0,0,0,0,0,121,4.0,0.0,0.0,16.0,20.0
T,0,0,0,0,0,0,0,0,173,12.0,0.0,8.0,0.0,20.0


In [6]:
# to compute 'high frequency' errors (e.g. SNPs and PCR errors combined) we do not need to look at
# stop codons, and also do not need to restrict to looking at only certain points in the genome
all_nuc_counts = df["REF"].value_counts()
x3 = df.where(df["ERROR_PC"] >= ERROR_CUTOFF).dropna()
x3["TO_A"] = x3.apply(lambda y: 1 if y.MUT_TO_A/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_A > 5 else 0 ,axis=1)
x3["TO_C"] = x3.apply(lambda y: 1 if y.MUT_TO_C/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_C > 5 else 0 ,axis=1)
x3["TO_G"] = x3.apply(lambda y: 1 if y.MUT_TO_G/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_G > 5 else 0 ,axis=1)
x3["TO_T"] = x3.apply(lambda y: 1 if y.MUT_TO_T/y.DEPTH > ERROR_CUTOFF and y.MUT_TO_T > 5 else 0 ,axis=1)

x3 = x3[["REF", "TO_A", "TO_C", "TO_G", "TO_T"]]
x3 = x3.groupby("REF").sum()
hifreq_errors = pd.merge(x3, all_nuc_counts, left_index=True, right_index=True)
hifreq_errors = hifreq_errors.rename(columns={"REF": "NUC_COUNT"})
hifreq_errors.head()

,TO_A,TO_C,TO_G,TO_T,NUC_COUNT
C,1,0,0,3,95
G,0,0,0,0,121
T,0,0,0,0,173


In [7]:
# whereas pcr and high frequency errors need to be computed per site, sequencing errors are
# expected to be independant between reads and so we need to normalise by depth. 
# hence the calculation here is somewhat different. 
# We will still save the columns to do with stop codons just out of interest but they will probably not be used
# in the final analysis.

x4 = df.where(df["ERROR_PC"] < ERROR_CUTOFF).dropna()

all_stop_codon_stop_counts = x4[["REF","DEPTH", "A_STOP", "C_STOP", "G_STOP", "T_STOP", "ANY_STOP"]]
all_stop_codon_stop_counts["A_STOP"] = all_stop_codon_stop_counts["A_STOP"] * all_stop_codon_stop_counts["DEPTH"]
all_stop_codon_stop_counts["A_STOP"] = all_stop_codon_stop_counts.apply(lambda x: int(x.REF != "A") * x.A_STOP, axis=1)

all_stop_codon_stop_counts["C_STOP"] = all_stop_codon_stop_counts["C_STOP"] * all_stop_codon_stop_counts["DEPTH"]
all_stop_codon_stop_counts["C_STOP"] = all_stop_codon_stop_counts.apply(lambda x: int(x.REF != "C") * x.C_STOP, axis=1)

all_stop_codon_stop_counts["G_STOP"] = all_stop_codon_stop_counts["G_STOP"] * all_stop_codon_stop_counts["DEPTH"]
all_stop_codon_stop_counts["G_STOP"] = all_stop_codon_stop_counts.apply(lambda x: int(x.REF != "G") * x.G_STOP, axis=1)

all_stop_codon_stop_counts["T_STOP"] = all_stop_codon_stop_counts["T_STOP"] * all_stop_codon_stop_counts["DEPTH"]
all_stop_codon_stop_counts["T_STOP"] = all_stop_codon_stop_counts.apply(lambda x: int(x.REF != "T") * x.T_STOP, axis=1)

all_stop_codon_stop_counts["ANY_STOP"] = all_stop_codon_stop_counts.apply(lambda x: int(x.A_STOP + x.C_STOP + x.G_STOP + x.T_STOP), axis=1)
all_stop_codon_stop_counts.drop(columns=["DEPTH"], inplace=True)
all_stop_codon_stop_counts = all_stop_codon_stop_counts.groupby("REF").sum()




x5 = x4[["REF","DEPTH", "MUT_TO_A", "MUT_TO_C", "MUT_TO_G", "MUT_TO_T"]]
x5["TO_A_STOP"] = x4.apply(lambda x: int(x.A_STOP * x.MUT_TO_A), axis=1)
x5["TO_C_STOP"] = x4.apply(lambda x: int(x.C_STOP * x.MUT_TO_C), axis=1)
x5["TO_G_STOP"] = x4.apply(lambda x: int(x.G_STOP * x.MUT_TO_G), axis=1)
x5["TO_T_STOP"] = x4.apply(lambda x: int(x.T_STOP * x.MUT_TO_T), axis=1)

seq_errors = x5.groupby("REF").sum()
seq_errors = pd.merge(seq_errors, all_stop_codon_stop_counts, left_index=True, right_index=True)
seq_errors = seq_errors.rename(
    columns={"DEPTH": "NUC_COUNT", "MUT_TO_A":"TO_A", "MUT_TO_C":"TO_C", "MUT_TO_G":"TO_G", "MUT_TO_T":"TO_T"})
seq_errors.head()

<ipython-input-1-683580d98072>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_stop_codon_stop_counts["A_STOP"] = all_stop_codon_stop_counts["A_STOP"] * all_stop_codon_stop_counts["DEPTH"]
<ipython-input-1-683580d98072>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_stop_codon_stop_counts["A_STOP"] = all_stop_codon_stop_counts.apply(lambda x: int(x.REF != "A") * x.A_STOP, axis=1)
<ipython-input-1-683580d98072>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,NUC_COUNT,TO_A,TO_C,TO_G,TO_T,TO_A_STOP,TO_C_STOP,TO_G_STOP,TO_T_STOP,A_STOP,C_STOP,G_STOP,T_STOP,ANY_STOP
REF,,,,,,,,,,,,,,
A,2312.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,136.0,136
C,1194.0,0.0,0.0,0.0,0.0,0,0,0,0,50.0,0.0,37.0,179.0,266
G,1545.0,0.0,0.0,0.0,0.0,0,0,0,0,48.0,0.0,0.0,195.0,243
T,2318.0,0.0,0.0,0.0,0.0,0,0,0,0,145.0,0.0,106.0,0.0,251


In [8]:
# save all of the files
df.to_csv(f"../data/ww_dump/dataframes/vcf_dataframe__{FILENAME}.csv", sep="\t")

In [9]:
seq_errors.to_csv(f"../data/ww_dump/dataframes/seqerrors__{FILENAME}.csv", sep="\t")

In [10]:
pcr_errors.to_csv(f"../data/ww_dump/dataframes/pcrerrors__{FILENAME}.csv", sep="\t")

In [11]:
hifreq_errors.to_csv(f"../data/ww_dump/dataframes/hifreqerrors__{FILENAME}.csv", sep="\t")